In [ ]:
# | default_exp agent.planner.executor

In [1]:
# | export
import json
from typing import Optional
from rich.console import Console

from agentic.core.agent import Agent, AgentConfig
from agentic.configs.loader import get_model_config

from agentic.agent.planner.models import ProjectContext, ExecutionStatus
from agentic.agent.planner.breakdown import ProjectBreakdownGenerator
from agentic.agent.planner.task_generator import TaskGenerator
from agentic.agent.planner.task_executor import TaskExecutor
from agentic.agent.planner.validation import TaskValidator
from agentic.agent.planner.cache import CacheManager

2025-09-30 11:28:28,425 - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-30 11:28:28,430 - INFO - Initialized LLM client with model: gpt-oss:20b
2025-09-30 11:28:28,519 - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-30 11:28:28,520 - INFO - Initialized LLM client with model: gpt-oss:20b


📋 Decomposing into tasks...

2025-09-30 11:28:29,425 - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"



╭─────────────────────── 🤔 Thinking ───────────────────────╮
│ We need to produce a JSON array of tasks with detailed substeps, introspection, dependencies, etc. First task is Requirement Analysis. Use code_interpreter or introspect. Also possibly debate_agent if frameworks chosen.

We only have one user prompt; we must propose tasks. This is a long response but must produce JSON. Need to structure tasks step by step.

We should think of high

KeyboardInterrupt: 

In [ ]:
# | export

class DynamicTaskExecutor:
    """Production-ready dynamic task execution system"""
    
    def __init__(self, model: str = None):
        # Load model from config if not provided
        if model is None:
            model_config = get_model_config()
            model = model_config.get('name', 'qwen3:8b')
        self.model = model
        config = AgentConfig(
            name="dynamic_executor",
            instructions="You are an AI agent that executes tasks systematically. Analyze requirements, break down complex problems, and provide detailed technical responses with proper implementation.",
            model=model,
            tools=["fs_read", "fs_write", "execute_bash"],
            temperature=0.7
        )
        self.agent = Agent(config)
        self.console = Console()
        
        # Initialize components
        self.breakdown_generator = ProjectBreakdownGenerator(self.agent, self.console)
        self.task_generator = TaskGenerator(self.agent, self.console)
        self.task_executor = TaskExecutor(self.agent, self.console)
        self.validator = TaskValidator(self.console)
        self.cache_manager = CacheManager(self.console)
        
        # Configuration
        self.max_retries = 3
        self.max_tasks = 20
        
        # State
        self.context: Optional[ProjectContext] = None
        self.project_breakdown = None
        self.estimated_total_tasks: int = 0
    
    async def execute_project(self, user_request: str) -> ProjectContext:
        """Main execution loop with two-phase approach"""
        self.console.print(f"🚀 Starting dynamic execution for: {user_request}")
        
        # Try to restore from cache
        restored_context, restored_breakdown = self.cache_manager.restore_from_cache(user_request)
        if restored_context and restored_breakdown:
            self.console.print("📦 Restored from cache, continuing execution...")
            self.context = restored_context
            self.project_breakdown = restored_breakdown
            self.estimated_total_tasks = self.breakdown_generator.estimate_total_tasks(self.project_breakdown)
        else:
            # Phase 1: Generate project breakdown
            self.console.print("\n📋 PHASE 1: Generating project breakdown...")
            self.project_breakdown = self.breakdown_generator.generate_project_breakdown(user_request)
            
            if not self.project_breakdown:
                self.console.print("❌ Failed to generate project breakdown")
                return ProjectContext(original_request=user_request, project_status=ExecutionStatus.FAILED)
            
            self.console.print(f"✅ Project breakdown complete")
            
            # Initialize project context
            self.context = ProjectContext(original_request=user_request)
            
            # Estimate total tasks from breakdown
            self.estimated_total_tasks = self.breakdown_generator.estimate_total_tasks(self.project_breakdown)
            
            # Save initial state
            self.cache_manager.save_to_cache(self.context, self.project_breakdown)
        
        # Phase 2: Execute tasks one by one
        self.console.print("\n🔄 PHASE 2: Executing tasks dynamically...")
        
        task_counter = len(self.context.execution_history) + 1
        
        while not self._is_project_complete() and task_counter <= self.max_tasks:
            self.console.print(f"\n{'='*60}")
            self.console.print(f"📋 GENERATING TASK {task_counter}")
            self.console.print(f"{'='*60}")
            
            # Generate next task using breakdown context
            task = self.task_generator.generate_next_task(self.context, self.project_breakdown, self.estimated_total_tasks)
            if not task:
                self.console.print("❌ No more tasks to generate. Project complete.")
                break
            
            self.console.print(f"✅ Generated: {task.name}")
            self.console.print(f"🔍 Task ID: {task.id}, Dependencies: {task.dependencies}")
            
            # PRE-EXECUTION INTROSPECTION
            pre_validation = self.validator.introspect_task_planning(task, self.project_breakdown, self.context)
            if not pre_validation.success:
                self.console.print(f"⚠️ Pre-execution validation failed: {pre_validation.feedback}")
                if pre_validation.next_action == "regenerate":
                    self.console.print("🔄 Regenerating task with feedback...")
                    regenerated_task = self.task_generator.regenerate_task_with_feedback(self.context, self.project_breakdown, pre_validation.feedback)
                    if regenerated_task:
                        task = regenerated_task
                        self.console.print("✅ Task regenerated successfully")
                    else:
                        self.console.print("⚠️ Task regeneration failed. Proceeding with original task.")
            
            # Execute task
            task_result = await self.task_executor.execute_task(task)
            
            # Update context
            self.context.execution_history.append(task_result)
            
            if not isinstance(self.context.current_artifacts, list):
                self.context.current_artifacts = []
                
            artifacts = task_result.artifacts_created
            if isinstance(artifacts, list):
                self.context.current_artifacts.extend(artifacts)
            elif artifacts:
                self.context.current_artifacts.extend([str(artifacts)])
            
            self.context.total_tasks_completed += 1
            
            # Save state after each task
            self.cache_manager.save_to_cache(self.context, self.project_breakdown)
            
            # Update project status
            if task_result.status == ExecutionStatus.FAILED:
                self.console.print(f"⚠️ Task {task.id} failed. Continuing with next task.")
                # Mark task as failed but don't stop execution
                failed_tasks = getattr(self.context, 'failed_tasks', [])
                failed_tasks.append(task.id)
                self.context.failed_tasks = failed_tasks
            
            task_counter += 1
        
        # Final project status
        failed_tasks = getattr(self.context, 'failed_tasks', [])
        if failed_tasks:
            self.context.project_status = ExecutionStatus.PARTIAL_SUCCESS
            self.console.print(f"⚠️ Project completed with {len(failed_tasks)} failed tasks: {failed_tasks}")
        else:
            self.context.project_status = ExecutionStatus.SUCCESS
            self.console.print("✅ All tasks completed successfully")
        
        # Clear cache on completion
        self.cache_manager.clear_cache()
        
        self.console.print(f"\n🎉 Project execution completed! Generated {len(self.context.execution_history)} tasks.")
        return self.context
    
    def _is_project_complete(self) -> bool:
        """Determine if project is complete based on progress"""
        if not self.context.execution_history:
            return False
        
        if self.context.total_tasks_completed >= max(self.estimated_total_tasks, 5):
            self.console.print(f"✅ Project completion reached: {self.context.total_tasks_completed}/{self.estimated_total_tasks} tasks")
            return True
        
        completion_percentage = (self.context.total_tasks_completed / max(self.estimated_total_tasks, 1)) * 100
        self.console.print(f"📊 Project progress: {self.context.total_tasks_completed}/{self.estimated_total_tasks} tasks ({completion_percentage:.1f}%)")
        
        return False
    
    def save_execution_report(self, filename: str = None) -> str:
        """Save detailed execution report"""
        if not filename:
            filename = "execution_report.json"
        
        with open(filename, 'w') as f:
            json.dump(self.context.model_dump(), f, indent=2, default=str)
        
        self.console.print(f"📊 Execution report saved to: {filename}")
        return filename
